In [ ]:
import geopandas as gpd
import json
import math
import numpy as np
import os
import pandas as pd
import plotly.express as px
import shutil as sh


# PARAMETERS
dataFolderName = 'data'
geoJsonFolder = dataFolderName+'/geoJson/'
figureOutputFolder = 'exported_figures'

In [ ]:
# Utility functions to read data from csv and remove useless columns frome the dataframe

def loadDataFromCSV():
    '''
    Load data about salaries into the program.
    Returns a Pandas Dataframe.
    '''

    dataFileName = dataFolderName + '/DCSC_RACLI_01092021113430630.csv'
    df = pd.read_csv(dataFileName).drop_duplicates()

    # Transform the data and remove useless columns
    df['Territorio'] = df['Territorio'].str.replace(' / ','/')
    df = df.drop('TIPO_DATO7', axis=1) # always the same (HOUWAG_ENTEMP_AV_MI)
    df = df.drop('Tipo dato', axis=1)  # always the same (Retribuzione lorda oraria per ora retribuita delle posizioni lavorative dipendenti in euro (media).)
    df = df.drop(['SEXISTAT1', 'ETA1_A','PROFILO_PROF','CLLVT','Seleziona periodo'], axis=1)  # ridondance of information
    df = df[df['Flag Codes'] != 'c'].drop(['Flags','Flag Codes'], axis=1) # delete incomplete data and drop columns with corresponding flag ('c' is the flag for hidden data)

    # Transform data for consistency with datasets of geocoords
    df.loc[df['Territorio']=="Forlì-Cesena", "Territorio"] = "Forli'-Cesena"

    return df


def getDataAboutTerritory():
    '''
    Returns data about salaries in territories (data about sectors are excluded).
    '''
    df = loadDataFromCSV().query('`Ateco 2007`=="TOTALE"').drop(['Ateco 2007', 'ATECO_2007'], axis=1)

    # Categorization of Salary values (grouping in categories)
    valueCountedData = np.floor(df["Value"]).astype(int).value_counts().sort_index(ascending=False)
    # groupedData = { floored: {territory: value for territory, value in df[['Territorio', 'Value']].to_numpy() if math.floor(value)==floored}    \
    #                     for floored in np.floor(df["Value"]).astype(int).drop_duplicates() }
    # print("Floor\tCount\n" + str(valueCountedData))
    # print(str(groupedData).replace('},',']\n'))
    # NOTE: This part should be part of data transforming? But ranges should adapt to the context?

    oldCategory=0
    # minValue = math.floor(df['Value'].min())
    # maxValue = math.ceil(df['Value'].max())
    # nCategories = 5
    # salaryCategoryBorders = range(minValue, maxValue+1, math.ceil((maxValue-minValue)/nCategories)) # nCategories categories
    salaryCategoryBorders = range(9,20,2)
    for category in salaryCategoryBorders:
        numberProvinceInThisCategory = sum([valueCountedData[key] for key in np.intersect1d(valueCountedData.keys().tolist(), range(oldCategory,category))])
        df.loc[(oldCategory<=df['Value']) & (df['Value']<category), "SalaryCategory"] =                 \
            (f"{oldCategory} ≤ " if oldCategory > salaryCategoryBorders[0] else "        ")             \
            + ".."                                                                                      \
            + (f" < {category}"  if category < salaryCategoryBorders[-1] else "        ")               \
            + f"  €/hr\t({numberProvinceInThisCategory} provinces)"
        oldCategory = category


    # sort (needed to respect the range-scale in plots if categorization is used)
    #   Sort (first) ascending wrt 'TIME' (oldest first) then descending wrt 'Value'
    df['Value'] = -df['Value']  # invert sign, so 'Value' can be sorted descending
    df.sort_values(by=['TIME', 'Value'], ascending=True, inplace=True)
    df['Value'] = -df['Value']  # restore the correct sign

    return df


def getDataAboutProvinces():
    '''
    Returns data about salaries in provinces (data about sectors, regions, entire Italy are excluded).
    '''
    df_territory = getDataAboutTerritory()
    return df_territory[df_territory['ITTER107'].str.contains('.{5}')].drop('ITTER107', axis=1)   # for provinces, 'ITTER107' code is 5 chars long


def getDataAboutProvincesInDictHavingYearsAsKey(years):
    '''
    Returns data about salaries in provinces (data about sectors, regions, entire Italy are excluded),
    organized in a dictionary having years (the parameters) as keys.
    Params: years, e.g.: years=range(2014,2018)
    '''
    return {year: getDataAboutProvinces().query(f'TIME=={year}').drop('TIME', axis=1).drop_duplicates() for year in years}


def getProvinceSalaryvalue(year):
    '''
    Returns a Pandas Dataframe with two columns: one for Province names and the second for corresponding salary values.
    Returned data refer to the year which is given as parameter.
    '''
    return getDataAboutProvincesInDictHavingYearsAsKey([year])[year]                                                                            \
                .query("Sesso=='totale' & `Classe di età`=='totale' & `Qualifica contrattuale`=='totale' & `Classe di dipendenti`=='totale'")   \
                .drop(['Sesso', 'Classe di età', 'Qualifica contrattuale', 'Classe di dipendenti'], axis=1)

   
def avgSalary(territory='Italia', year=-1):
    '''
    Returns the average salary value in a given territory for a given year (parameters).
    The default value for the territory is entire Italy.
    If the year is not specified, the average value is computed over all the years which
    are available.
    '''
    query = f"Territorio=='Italia' & Sesso=='totale' & `Classe di età`=='totale' & `Qualifica contrattuale`=='totale' & `Classe di dipendenti`=='totale'"   \
            + (f" & `TIME=={year}" if year!=-1 else "")
    return round(100*getDataAboutTerritory().query(query)['Value'].mean())/100  # round(100*..)/100 is used to have two decimal digits

In [ ]:
# Utility functions for geo-data
def readGeoDataToDictHavingYearAsKey():
    '''
    Import data Geo-data (coordinates) and returns the dictionary having as key
    the year and as values the dataframe with geodata loaded from shape files.
    '''
    map_df = {} # dictionary, year as key
    map_df[2014] = gpd.read_file(f'{dataFolderName}/province_shapes/Prov01012014_g/Prov01012014_g_WGS84.shp')
    map_df[2014]['DEN_PCM'] = map_df[2014]['DEN_PROV']  # duplicate this column to make the dataframe compliant with those of subsequent years 
    map_df[2014].loc[ map_df[2014].DEN_PCM=="Forlì-Cesena","DEN_PCM" ] = "Forli'-Cesena"

    for year in range(2015,2018):
        fp = f'{dataFolderName}/province_shapes/ProvCM01012017_g/ProvCM01012017_g_WGS84.shp' # data updated to 1st Jan 2017 work for our purposes
        map_df[year] = gpd.read_file(fp) #reading the file stored in variable fp
        map_df[year].loc[ map_df[year].DEN_PCM=="Aosta","DEN_PCM" ] = "Valle d'Aosta/Vallée d'Aoste"
        map_df[year].loc[ map_df[year].DEN_PCM=="Massa Carrara","DEN_PCM" ] = "Massa-Carrara"
        map_df[year].loc[ map_df[year].DEN_PCM=="Bolzano","DEN_PCM" ] = "Bolzano/Bozen"
    
    return map_df

# Function to convert (project) coordinates to latitude/longitude
def convertCrsToLatLong(inputGeopandasDf, inplace=False):
    '''
    Convert the geo-coordinates of the iunput GeoPandas Dataframe to EPSG:4326 (latitude and longitude)
    and returns a new GeoPandas dataframe having the data in the new coordinates system.
    You can specify the parameter inplace=True if you want to change the coordinate system "inplace",
    i.e., directly in the input GeoPandas Dataframe.
    '''
    outputGeopandasDf = inputGeopandasDf.set_geometry("geometry") # The original geometry column is replaced with "geometry" (if it was different).
    outputGeopandasDf = outputGeopandasDf.to_crs("EPSG:4326", inplace=inplace)
    return outputGeopandasDf

def createGeoJsonFromFile(geoJsonFolder, shapeDataDictYears, convertCrsToLatLongFlag=True):
    '''
    Creates GeoJson files in the folder whose path is specified as parameter as string,
    from the given dictionary having years as keys and the corresponding shape file data
    (GeoPandas dataframe) as values.
    Specify the parameter convertCrsToLatLongFlag=False if you do NOT want to convert the
    geo-coordinate system to EPSG:4326; default is True.
    Returns a dictionary having as key the years (the same as the input dictionary) and
    the corresponding GeoJson data as values.
    '''
    geoJsonData = {}
    if not os.path.exists(geoJsonFolder):
        os.makedirs(geoJsonFolder)
    for year in shapeDataDictYears:
        if(convertCrsToLatLongFlag):
            shapeDataDictYears[year] = convertCrsToLatLong(shapeDataDictYears[year])
        geoJsonPathThisYear = geoJsonFolder+str(year)+'.json'
        shapeDataDictYears[year].to_file(geoJsonPathThisYear, driver="GeoJSON")
        with open(geoJsonPathThisYear, encoding="utf-8") as geofile:
            geoJsonData[year] = json.load(geofile)
    
    return geoJsonData

In [ ]:
# Create geo-data, year by year, correcting input data to make them compatible with our data

map_df = readGeoDataToDictHavingYearAsKey() # dictionary, year as key

years = tuple(map_df.keys())
print('Input Coordinate Reference System: \t ' + str(map_df[years[0]].crs)) # print the Coordinate Reference System (CRS), EPSG:32632 is WGS 84 / UTM zone 32N (WGS=World Geodetic System, UTM=Universal Transverse Mercator)

# Create a GeoJson file from the *.shp and read it
geoJsonData = createGeoJsonFromFile(geoJsonFolder, map_df)
print('GeoJson Coordinate Reference System: \t ' + str(map_df[years[0]].crs) + '\n') # print the Coordinate Reference System (CRS)

# Plot the map (unnecessary, just to show that it works)
map_df[years[0]].plot()

print(map_df[years[0]].head())  # NOTE: 'DEN_PCM' COLUMNS contains distinct values
for year in map_df:
    print(f'Year {year}:\t' + str(len(map_df[year])) + ' ' + str(len(map_df[year]['DEN_PCM'].drop_duplicates())))
    # # Print duplicates:
    # print(map_df[year] [map_df[year].duplicated('DEN_PCM')])

In [ ]:
# Load data about provinces
df_province = getDataAboutProvinces()
# print(df_province["Territorio"].drop_duplicates().sort_values().tolist())
print(df_province.head())

# Create a dictionary having years as keys
df_province_years = getDataAboutProvincesInDictHavingYearsAsKey(years)
# print(pd.DataFrame(df_province_years[years[0]]).head())

print(f"Average salary in Italy over years: " + str(avgSalary()) + " €/hr")

In [ ]:
# Test data for the map
test_year = 2017        # year to use for tests
df_test = getProvinceSalaryvalue(test_year)
geoJsonData_test = geoJsonData[test_year]

print(df_province_years[test_year]["Territorio"].drop_duplicates().sort_values().tolist())
print()
print(map_df[test_year]["DEN_PCM"].drop_duplicates().sort_values().tolist())
print()
print('\n\t\tproperties.DEN_PCM:\n' + str(sorted([prov['properties']['DEN_PCM'] for prov in geoJsonData_test['features']])) + '\n\n')

print(df_test.head())
print(geoJsonData_test.keys())
print('\tfeatures:\t' + str(geoJsonData_test['features'][0].keys()))
print('\t\tproperties:\t' + str(geoJsonData_test['features'][0]['properties'].keys()))

print(geoJsonData_test['features'][0]['properties'].values())
# print(geoJsonData_test['features'][0]['geometry']['coordinates'])
print(geoJsonData_test['features'][1]['properties'].values())


print('Same lenght of dataframes? ' + str(len(df_test)==len(geoJsonData_test['features'])) + ' (' + str(len(df_test)) + ')')

print(geoJsonData_test['features'][0]['properties'].values())

In [ ]:
# Choropleth by categories

fig = px.choropleth(
    data_frame=df_test, 
    geojson=geoJsonData_test, 
    locations='Territorio',              # name of dataframe column
    featureidkey='properties.DEN_PCM',   # path to field in GeoJSON feature object with which to match the values passed in to locations
    color='SalaryCategory',
    # color_continuous_scale="rdylgn",                                                                          # for continuos scale of colors
    color_discrete_sequence=['#4dac26', '#b8e186', '#fefee9', '#f1b6da', '#d01c8b'],      # for discrete scale of colors
    center={"lat": 42, "lon": 13},
    projection='mercator',
    labels={'SalaryCategory': 'Average hourly gross salary'},
    hover_name='Territorio',
    hover_data={'Value':True, 'SalaryCategory':False, 'Territorio': False}          # TODO: improve this (see "hovertemplate")
)
fig.update_traces(marker=dict(opacity=1, line=dict(color='black', width=0.1)))      # TODO: look for "hovertemplate, https://plotly.com/python/reference/choropleth/#choropleth-hovertemplate"
fig.update_layout(plot_bgcolor='white',
    font=dict(color='dimgray'),
    title='Salaries in private companies',
    margin={"r":0,"t":0,"l":0,"b":0},
    legend_itemsizing='trace'               # Determines if the legend items symbols scale with their corresponding "trace" attributes or remain "constant" independent of the symbol size on the graph. # TODO: NOT working
)
fig.update_geos(showcountries=False, showcoastlines=False, showland=False, fitbounds="locations")
fig.show("notebook")



# # Export the figure
# if os.path.exists(figureOutputFolder):
#     sh.rmtree(figureOutputFolder)

# os.makedirs(figureOutputFolder)
# fig.write_image(f"{figureOutputFolder}/testGeoMap{test_year}.svg")
# fig.write_image(f"{figureOutputFolder}/testGeoMap{test_year}.png", width=3000, height=2000)
# fig.to_image(format="png", width=1920, height=1080, scale=2).write(f"testGeoMap{test_year}_HighResolution.png")


# TODO : check data values
# TODO : legend symbol sizes proportional to number of provinces in the category
# TODO : show (infografic or similar) best and worst province salary and/or values of biggest cities (Milan, Rome, Turin, Naples, Palermo, Venice, Trieste)
# TODO : show data divided by principal;employeers / workers / apprentinces

In [ ]:
# Choropleth mapbox
fig = px.choropleth_mapbox(
    df_test,
    geojson=geoJsonData_test,       
    color='Value',                      # name of a dataframe column
    locations='Territorio',             # name of a dataframe column
    featureidkey="properties.DEN_PCM",
    center={"lat": 42, "lon": 13},
    mapbox_style="carto-positron",
    zoom=4.4
)
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.update_geos(showcountries=False, showcoastlines=False, showland=False, fitbounds="locations")
fig.show()

In [ ]:
# NOTE : CODE DUPLICATION!

# Create geo-data, year by year, correcting input data to make them compatible with our data
map_df = {} # dictionary, year as key
map_df[2014] = gpd.read_file(f'{dataFolderName}/province_shapes/Prov01012014_g/Prov01012014_g_WGS84.shp')
map_df[2014]['DEN_PCM'] = map_df[2014]['DEN_PROV']  # duplicate this column to make the dataframe compliant with those of subsequent years 
map_df[2014].loc[ map_df[2014].DEN_PCM=="Forlì-Cesena","DEN_PCM" ] = "Forli'-Cesena"

for year in range(2015,2018):
    fp = f'{dataFolderName}/province_shapes/ProvCM01012017_g/ProvCM01012017_g_WGS84.shp' # data updated to 1st Jan 2017 work for our purposes
    map_df[year] = gpd.read_file(fp) #reading the file stored in variable fp
    map_df[year].loc[ map_df[year].DEN_PCM=="Aosta","DEN_PCM" ] = "Valle d'Aosta/Vallée d'Aoste"
    map_df[year].loc[ map_df[year].DEN_PCM=="Massa Carrara","DEN_PCM" ] = "Massa-Carrara"
    map_df[year].loc[ map_df[year].DEN_PCM=="Bolzano","DEN_PCM" ] = "Bolzano/Bozen"

# Note: territories coords change over the year, hence we save the year near the territory names
df_province["TerritorioAnno"] = df_province["Territorio"] + df_province['TIME'].astype(str)
for year in range(2014, 2018):
    map_df[year]["TerritorioAnno"] = map_df[year]["DEN_PCM"] + str(year)


## Change the coordinate system, then create, save and load the GeoJson data

# Function to convert (project) coordinates to latitude/longitude
def convertCrsToLatLong(inputGeopandasDf):
    outputGeopandasDf = inputGeopandasDf.set_geometry("geometry") # The original geometry column is replaced with "geometry" (if it was different).
    outputGeopandasDf = outputGeopandasDf.to_crs("EPSG:4326")
    return outputGeopandasDf

# Union of geodata over years
map_df = pd.concat([convertCrsToLatLong(map_df[year]) for year in range(2014,2018)])

# Create a GeoJson file from the *.shp and read it
geoJsonPath = geoJsonFolder + '.json'
if not os.path.exists(geoJsonFolder):
    os.makedirs(geoJsonFolder)
map_df.to_file(geoJsonPath, driver="GeoJSON")
with open(geoJsonPath, encoding="utf-8") as geofile:
    geoJsonData = json.load(geofile)

print(df_province.head())
# print(geoJsonData)


# Choropleth with slider animation over years
fig = px.choropleth(
    data_frame=df_province, 
    geojson=geoJsonData, 
    locations='TerritorioAnno',                 # name of dataframe column
    hover_name='Territorio',
    hover_data={'Value':True, 'TerritorioAnno':False},
    featureidkey='properties.TerritorioAnno',   # path to field in GeoJSON feature object with which to match the values passed in to locations
    color='Value',
    color_continuous_scale="Magma",
    center={"lat": 42, "lon": 13},
    labels={'Value':'Salary in € ', 'TIME':'Year'},
    projection='mercator',
    animation_frame="TIME"
)
fig.layout.updatemenus[0].buttons[0].args[1]['frame']['duration'] = 1000
fig.layout.updatemenus[0].buttons[0].args[1]['transition']['duration'] = 100
fig.update_geos(showcountries=False, showcoastlines=False, showland=False, fitbounds="locations")
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show("notebook")

# # Export the figure
# fig.write_image(f"{figureOutputFolder}/testSliderGeoMap.svg")
# fig.write_image(f"{figureOutputFolder}/testSliderGeoMap.png")
# fig.write_html(f"{figureOutputFolder}/testSliderGeoMap_HighResolution.html")

In [ ]:
# NOTE : CODE DUPLICATION!

# Show map for only one province
provinceName = "Milano"

# Create geo-data, year by year, correcting input data to make them compatible with our data
map_df = readGeoDataToDictHavingYearAsKey()

# Note: territories coords change over the year, hence we save the year near the territory names
df_province["TerritorioAnno"] = df_province["Territorio"] + df_province['TIME'].astype(str)
for year in range(2014, 2018):
    map_df[year] = map_df[year][map_df[year]["DEN_PCM"]==provinceName]
    map_df[year]["TerritorioAnno"] = map_df[year]["DEN_PCM"] + str(year)


## Change the coordinate system, then create, save and load the GeoJson data

# Function to convert (project) coordinates to latitude/longitude
def convertCrsToLatLong(inputGeopandasDf):
    outputGeopandasDf = inputGeopandasDf.set_geometry("geometry") # The original geometry column is replaced with "geometry" (if it was different).
    outputGeopandasDf = outputGeopandasDf.to_crs("EPSG:4326")
    return outputGeopandasDf

# Union of geodata over years
map_df = pd.concat([convertCrsToLatLong(map_df[year]) for year in range(2014,2018)])

# Create a GeoJson file from the *.shp and read it
geoJsonPath = geoJsonFolder + '.json'
if not os.path.exists(geoJsonFolder):
    os.makedirs(geoJsonFolder)
map_df.to_file(geoJsonPath, driver="GeoJSON")
with open(geoJsonPath, encoding="utf-8") as geofile:
    geoJsonData = json.load(geofile)

print(df_province.head())
# print(geoJsonData)


# Choropleth with slider animation over years
fig = px.choropleth(
    data_frame=df_province, 
    geojson=geoJsonData, 
    locations='TerritorioAnno',                 # name of dataframe column
    hover_name='Territorio',
    hover_data={'Value':True, 'TerritorioAnno':False},
    featureidkey='properties.TerritorioAnno',   # path to field in GeoJSON feature object with which to match the values passed in to locations
    color='Value',
    color_continuous_scale="Magma",
    center={"lat": 42, "lon": 13},
    labels={'Value':'Salary in € ', 'TIME':'Year'},
    projection='mercator',
    animation_frame="TIME"
)
fig.layout.updatemenus[0].buttons[0].args[1]['frame']['duration'] = 1000
fig.layout.updatemenus[0].buttons[0].args[1]['transition']['duration'] = 100
fig.update_geos(showcountries=False, showcoastlines=False, showland=False, fitbounds="locations")
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show("notebook")